In [0]:
df = spark.read.option("header", True).csv("/FileStore/tables/players.csv")
df.printSchema()
df.show(5)

root
 |-- player_id: string (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- last_season: string (nullable = true)
 |-- current_club_id: string (nullable = true)
 |-- player_code: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- city_of_birth: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- date_of_birth: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- position: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: string (nullable = true)
 |-- contract_expiration_date: string (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- url: string (nullable = true)
 |-- current_club_domestic_competition_id: string (nullable = true)
 |-- current_club_name: string (nullable = true)
 |-- market_value_in_eur: string (nullable = true)
 |--

In [0]:
from pyspark.sql.functions import col, desc

# 1. Conversão de tipos
df = df.withColumn("market_value_in_eur", col("market_value_in_eur").cast("int"))

# 2. Filtro de registros válidos
df_valid = df.filter(col("market_value_in_eur").isNotNull() & (col("market_value_in_eur") > 0))

# 3. Top jogadores por nacionalidade
df_top = df_valid.groupBy("country_of_citizenship").agg(
    {"market_value_in_eur": "max"}
).withColumnRenamed("max(market_value_in_eur)", "max_value_eur")

df_top = df_top.orderBy(desc("max_value_eur"))

# 4. Exibir resultado
df_top.show(10)

# 5. Exportar resultado como Parquet
df_top.write.mode("overwrite").parquet("/FileStore/tables/top_jogadores_por_nacao.parquet")


+----------------------+-------------+
|country_of_citizenship|max_value_eur|
+----------------------+-------------+
|                Norway|    200000000|
|                Brazil|    200000000|
|                 Spain|    180000000|
|               England|    180000000|
|                France|    170000000|
|               Germany|    140000000|
|               Uruguay|    130000000|
|                Sweden|    100000000|
|             Argentina|     95000000|
|              Colombia|     85000000|
+----------------------+-------------+
only showing top 10 rows



In [0]:
df_top.write.mode("overwrite").option("header", True).csv("/FileStore/tables/top_jogadores_por_nacao_csv")


In [0]:
dbutils.fs.ls("/FileStore/tables/top_jogadores_por_nacao_csv/")


Out[6]: [FileInfo(path='dbfs:/FileStore/tables/top_jogadores_por_nacao_csv/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1747927846000),
 FileInfo(path='dbfs:/FileStore/tables/top_jogadores_por_nacao_csv/_committed_5941356486223486116', name='_committed_5941356486223486116', size=112, modificationTime=1747927846000),
 FileInfo(path='dbfs:/FileStore/tables/top_jogadores_por_nacao_csv/_started_5941356486223486116', name='_started_5941356486223486116', size=0, modificationTime=1747927845000),
 FileInfo(path='dbfs:/FileStore/tables/top_jogadores_por_nacao_csv/part-00000-tid-5941356486223486116-2838e9a4-b027-4956-9545-cf33c9710eef-17-1-c000.csv', name='part-00000-tid-5941356486223486116-2838e9a4-b027-4956-9545-cf33c9710eef-17-1-c000.csv', size=3291, modificationTime=1747927846000)]

In [0]:
# Caminho base
base_url = "https://community.cloud.databricks.com/files/top_jogadores_por_nacao_csv/"

# Listar arquivos
arquivos = dbutils.fs.ls("/FileStore/tables/top_jogadores_por_nacao_csv/")

# Filtrar só o CSV real
for arquivo in arquivos:
    if arquivo.name.endswith(".csv"):
        print("🔗 Link para download:")
        print(base_url + arquivo.name)


🔗 Link para download:
https://community.cloud.databricks.com/files/top_jogadores_por_nacao_csv/part-00000-tid-5941356486223486116-2838e9a4-b027-4956-9545-cf33c9710eef-17-1-c000.csv


In [0]:
# Salvar um único CSV em um único arquivo
df_top.coalesce(1).write.mode("overwrite").option("header", True).csv("/tmp/top_jogadores.csv")


dbutils.fs.ls("/tmp/top_jogadores.csv/")


# Copiar o arquivo CSV do /tmp para /FileStore para acesso via navegador
dbutils.fs.cp(
    "dbfs:/tmp/top_jogadores.csv/part-00000-tid-7642473000456351545-e8cec414-f7bb-4552-97d0-cf048f558e95-23-1-c000.csv",
    "dbfs:/FileStore/top_jogadores_por_nacao.csv"
)



Out[10]: True

In [0]:
display(df_top)


country_of_citizenship,max_value_eur
Norway,200000000
Brazil,200000000
Spain,180000000
England,180000000
France,170000000
Germany,140000000
Uruguay,130000000
Sweden,100000000
Argentina,95000000
Colombia,85000000
